In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from datetime import date

def getDataframe(jsonPath):
  with open(jsonPath) as f:
    data = json.load(f)

  #Create general data frame
  df = pd.DataFrame(columns=["user","likes", "followers", "item0", "item1", "date"])

  #Put the data in the data frame
  for i in range(0,len(data['output'])):
    df.loc[i] = [data['output'][i]["user"], data['output'][i]["likes"], data['output'][i]["followers"], data['output'][i]["item0"], data['output'][i]["item1"],data['output'][i]["date"]]

  #Change the type of followers and likes variables (String to int)
  df['followers']=df['followers'].astype(int)
  df['likes']=df['likes'].astype(int)
  df['date']=pd.to_datetime(df['date'])
  

  #Create new colum in panda series format in order to calculate difference in days
  today = pd.to_datetime("today")
  df['diff_days']= (today - df['date']).dt.days + 1
  df['f_score'] = 0.5

  return df



In [154]:
def getLabelsDF(df,item):
  """ Parameters: Dataframe, name ofº the column that contains the item. Example 'item0' """
  
  df_item = pd.DataFrame(columns=["item","counts"])
  j=0
  for i in range(0, len(df[item].value_counts())):
    df_item.loc[j]=df[item].value_counts().index.tolist()[i],df[item].value_counts()[i]
    j=j+1

  
  return df_item

In [155]:
if __name__ == "__main__":
  postsDF = getDataframe("media-output.json")
  topLabelsDF = getLabelsDF(postsDF,'item0')
  bottomLabelsDF = getLabelsDF(postsDF,'item1')


In [156]:
def engagementPost(df):
    # TODO: Define better formula
    # TODO: Change 36 to 9 and define better day mapping
    df['post_engagement'] = df['likes'] / df['followers'] * (36 - df['diff_days'])
    

def engagementUser(df):
    engagements = {}
    for i in range(0, len(df['user'])):
        usr = df['user'][i]
        if usr not in engagements:
            engagements[usr] = df['post_engagement'][i]
        else:
            engagements[usr] = engagements[usr] + df['post_engagement'][i] 
    
    
    for user in engagements:
        engagements[user] = round(engagements[user] / df['user'].value_counts()[user] , 2)
    print(engagements)
 
    '''dx = pd.DataFrame(list(engagements.items()), columns =['username','avg_engagement'])
    for i in range(0, len(dx['username'])):
        for k in range(0, len(df['user'])):
            if df['user'][k] == dx['username'][i]:
                df['user_engagement'][k]= dx['avg_engagement'][i]'''


    df['user_engagement'] = 0
    
    for i in range(0, len(df['user'])):
        usr = df['user'][i]
        df['user_engagement'][i] = engagements[usr]
    

engagementPost(postsDF)
engagementUser(postsDF)



{'traffygirls': 7.89, 'giuseppe': 17.14, 'annapenello': 9.61, 'ferran_marques': 4.51, 'mariapombo': 5.09, 'josefonollosa': 5.62, 'oscargonzalez': 7.15, 'tedesko': 10.58, 'sarda': 5.32, 'accenture': 5.57}


C:\Users\joser\AppData\Local\Temp/ipykernel_11396/982915721.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user_engagement'][i] = engagements[usr]


In [157]:
postsDF

,user,likes,followers,item0,item1,date,diff_days,f_score,post_engagement,user_engagement
0,traffygirls,7223,6832,long_sleeve_top green,skirt white,2021-10-20,30,0.5,6.343384,7
1,traffygirls,435,832,long_sleeve_outwear pink,shorts beige,2021-10-20,30,0.5,3.137019,7
2,traffygirls,8538,1832,long_sleeve_top yellow,skirt orange,2021-10-19,31,0.5,23.302402,7
3,giuseppe,6157,1832,long_sleeve_outwear purple,shorts yellow,2021-10-19,31,0.5,16.804039,17
4,giuseppe,14619,1832,long_sleeve_top green,skirt white,2021-10-19,31,0.5,39.899017,17
...,...,...,...,...,...,...,...,...,...,...
100,traffygirls,6157,6832,long_sleeve_outwear purple,shorts yellow,2021-10-19,31,0.5,4.506001,7
101,accenture,8110,1832,long_sleeve_outwear pink,shorts red,2021-10-16,34,0.5,8.853712,5
102,accenture,12429,10832,long_sleeve_top blue,skirt orange,2021-10-16,34,0.5,2.294867,5
103,tedesko,5245,3832,long_sleeve_outwear purple,shorts purple,2021-10-16,34,0.5,2.737474,10
